In [101]:
import re, requests, json

taskAnnotation = """
{
"result": [
            {allAnnotations}
        ],
        "ground_truth": false,
        "lead_time": 1
    }
"""

subtaskAnnotation = """
        {
          "original_length": {taskLen},
          "value": {
            "start": {taskStart},
            "end": {taskEnd},
            "labels": [
              "Speaker"
            ]
          },
          "id": "{taskID}",
          "from_name": "labels",
          "to_name": "audio",
          "type": "labels",
          "origin": "manual"
        },
        {
          "original_length": {taskLen},
          "value": {
            "start": {taskStart},
            "end": {taskEnd},
            "text": [
              "{taskTEXT}"
            ]
          },
          "id": "{taskID}",
          "from_name": "text",
          "to_name": "audio",
          "type": "textarea",
          "origin": "manual"
        },
        {
          "original_length": {taskLen},
          "value": {
            "start": {taskStart},
            "end": {taskEnd},
            "text": [
              "{taskASR}"
            ]
          },
          "id": "{taskID}",
          "from_name": "asr",
          "to_name": "audio",
          "type": "textarea",
          "origin": "manual"
        },
                {
          "original_length": {taskLen},
          "value": {
            "start": {taskStart},
            "end": {taskEnd},
            "text": [
              "{taskASR}"
            ]
          },
          "id": "{taskID}",
          "from_name": "asr1",
          "to_name": "audio",
          "type": "textarea",
          "origin": "manual"
        },
                        {
          "original_length": {taskLen},
          "value": {
            "start": {taskStart},
            "end": {taskEnd},
            "text": [
              "{taskTEXT}"
            ]
          },
          "id": "{taskID}",
          "from_name": "text1",
          "to_name": "audio",
          "type": "textarea",
          "origin": "manual"
        }
"""

import os
import random
import gzip, json
from pydub import AudioSegment
import re, boto3
import requests, datetime
from label_studio_sdk import Client
import os, sys

curr_sec = 0
sec = 1000
minute = 60 * sec

directory = 'Batch_1'
batch = '1'

sentences = {}

# Concatinate files

In [102]:
with open(directory + '/sentences.txt', 'r', encoding='utf8') as f:
  for l in f:
    p = l.strip().replace('\\t','\t').split('\t')
    if len(p) == 2:
      sentences[int(p[0])] = p[1]  


In [103]:
# ranges = [[1,301,1], [301,588,2]] # [startSentence, EndSentence+1, group_number]
ranges = [[32,43,1]]
for r in ranges:
  annotations = []
  outputAudio = AudioSegment.empty()
  outputAudio += AudioSegment.silent(duration=200)
  for i in range(r[0], r[1]):
    pad1 = 'ES000000'
    pad2 = '-normalized'
    if i < 10:
      pad = '0'
    if os.path.isfile('./' + directory + '/' + pad1 + str(i) + pad2 + '.wav'):
      print('./' + directory + '/' + pad1 + str(i) + pad2 + '.wav')
      data = AudioSegment.from_wav('./' + directory + '/' + pad1 + str(i) + pad2 + '.wav')
      segmentID = str(i)
      segmentLen = len(data)/sec
      startTime = len(outputAudio)/sec
      endTime = startTime + segmentLen
      text = text1 = sentences[i]
      asr = asr1 = 'asr'+ sentences[i]
      tmp = subtaskAnnotation.replace('{taskLen}', str(endTime - startTime)).replace('{taskStart}', str(startTime)).replace('{taskEnd}', \
                                       str(endTime)).replace('{taskID}', str(i)).replace('{taskASR}', asr.strip().replace('"', '')).replace('{taskTEXT}', text.strip().replace('"', ''))
      annotations.append(tmp)
      outputAudio += data
      outputAudio += AudioSegment.silent(duration=200)
  fullAnnotation = taskAnnotation.replace('{allAnnotations}', ',\n'.join(annotations))
  fullAnnotation = json.loads(fullAnnotation)
  outputFileName = directory + '/golara.batch_' + batch + '_' + str(r[2]) + '.wav'
  outputAudio.export(outputFileName, format='wav')
  with open(directory + '/golara.batch_' + batch + '_' + str(r[2]) + '.json', 'w', encoding='utf8') as f:
    f.write(json.dumps(fullAnnotation))

./Batch_1/ES00000032-normalized.wav
./Batch_1/ES00000033-normalized.wav
./Batch_1/ES00000034-normalized.wav
./Batch_1/ES00000035-normalized.wav
./Batch_1/ES00000036-normalized.wav
./Batch_1/ES00000037-normalized.wav
./Batch_1/ES00000038-normalized.wav
./Batch_1/ES00000039-normalized.wav
./Batch_1/ES00000040-normalized.wav
./Batch_1/ES00000041-normalized.wav
./Batch_1/ES00000042-normalized.wav


# Upload data to LabelStudio

In [104]:
import os
import random
import gzip, json
from pydub import AudioSegment
import re, boto3
import requests, datetime
from label_studio_sdk import Client

# upload data to label studio
def postAnnotations(url, annotationDraft, headers, taskID):
    url_annotate = url + str(taskID) + '/annotations/'
    response = requests.post(url=url_annotate, data=annotationDraft, headers=headers)
    return response.content

In [105]:
def postAnnotation(annotationDraft, headers, taskID):
    url_annotate = 'https://label.aixplain.com/api/tasks/' + str(taskID) + '/annotations/'
    response = requests.post(url=url_annotate, data=annotationDraft, headers=headers)
    return response.content

In [106]:
auth_token = '7fbe7f2b6e690b9d63076e16839a713012f25f5a'
url = 'https://label.aixplain.com/api/tasks/'
headers = {'Content-Type': 'application/json', 'Authorization': f'Token {auth_token}'}

for r in ranges:
    current_output_wav_file = 'golara.batch_' + batch + '_' + str(r[2]) + '.wav'

    filesToUpload = open('out.wav.segmentFiles.Batch_' + batch + '_' + str(r[2]) + '.csv', mode='w', encoding='utf8')
    filesToUpload.write('audio,label,url\n')
    filesToUpload.write('https://aix-label-studio.s3.amazonaws.com/TTS/Spanish/' + current_output_wav_file + ',1,https://aix-label-studio.s3.amazonaws.com/TTS/Spanish/' + current_output_wav_file)
    filesToUpload.close()

    #Upload to Label Studio to upload project
    url_annotate = 'https://label.aixplain.com/'
    project_id = 1259
    ls = Client(url=url_annotate, api_key=auth_token, credentials=auth_token)
    created_project = ls.get_project(project_id)
    task_id = created_project.import_tasks('out.wav.segmentFiles.Batch_' + batch + '_' + str(r[2]) + '.csv')
    print(task_id)
    with open(directory + '/golara.batch_' + batch + '_' + str(r[2]) + '.json', 'r', encoding='utf8') as f:
        fullAnnotation = json.loads(f.read())
    response = postAnnotation(json.dumps(fullAnnotation), headers, task_id[0])

[361218]
